In [1]:
import requests

In [73]:
import minsearch
from openai import OpenAI
import json 
from elasticsearch import Elasticsearch
from loguru import logger
import os 
from groq import Groq 
from dotenv import load_dotenv 
load_dotenv()

client = Groq(api_key = os.environ.get('GROQ_API_KEY'))

In [3]:
es_client = Elasticsearch(
    ['http://localhost:9200'],
    headers={
        "Accept": "application/vnd.elasticsearch+json; compatible-with=8",
        "Content-Type": "application/json"
    }  
)

Q1. Running Elastic

In [4]:
!curl http://localhost:9200  

{
  "name" : "fce58b346fab",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "Qq4UmIVQQSOlGEqarGRzdA",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
# What's the version.build_hash value?

"build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7"

Getting the Data

In [5]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.


Which function do you use for adding your data to elastic?


**Answer** : index


In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"



# Delete if exists
if es_client.indices.exists(index=index_name, body=index_settings):
    es_client.indices.delete(index=index_name, body=index_settings)

# Recreate the index

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [10]:
from tqdm.auto import tqdm 

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [ ]:
Which function do you use for adding your data to elastic?


**Answer** : index


In [34]:
index = minsearch.Index(
    text_fields=["question" , "text" ,"section"],
    keyword_fields= ["course"]
)

In [37]:
index.fit(documents)

Q3. Searching
Now let's search in our index.

We will execute a query "How do execute a command on a Kubernetes pod?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

84.50
64.50
44.50
24.50
Look at the _score field.

### Method-I use minisearch

In [63]:
query = 'How do execute a command on a Kubernetes pod?'

In [64]:
def search(query):
    boost = {'question': 4.0 , 'section' : 0.5}

    results = index.search(
        query = query,
        boost_dict = boost,
        num_results = 5
    )

    return results 

In [65]:
search(query)

[{'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
  'section': '10. Kubernetes and TensorFlow Serving',
  'question': 'Kubernetes-dashboard',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'In Kubernetes resource specifications, such as CPU requests and limits, the "m" stands for milliCPU, which is a unit of computing power. It represents one thousandth of a CPU core.\ncpu: "100m" means the container is requesting 100 milliCPUs, which is equivalent to 0.1 CPU core.\ncpu: "500m" means the container has a CPU limit of 500 milliCPUs, which is equivalent to 0.5 CPU core.\nThese values are specified in milliCPUs to allow fine-grained control over CPU resources. It allows you to express CPU requirements and limits in a more granular way, especially in scenarios where your application might not need a full CPU core.\nAdded by Andrii Larkin',
  'section': '10. Kubernetes and TensorFlow Serving',
  'question': 'Why cpu vals for Kubernetes deployment.yaml look like “100m” and “5

In [66]:
results = search(query)

print(results[0])

{'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 'question': 'Kubernetes-dashboard', 'course': 'machine-learning-zoomcamp'}


### Method: 2 Use ElasticSearch

In [74]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section^0.5"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index= index_name, body= search_query)
    logger.info(f"Top score:- {response['hits']['hits'][0]['_score']}")
    
    result_doc = []

    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])

    return result_doc

In [75]:
elastic_search(query)

2025-06-11 19:51:18.369 | INFO     | __main__:elastic_search:19 - Top score:, 44.50556


[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
  'section': '10. Kubernetes and TensorFlow Serving',
  'question': 'Kubernetes-dashboard',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t

In [ ]:
What's the score for the top ranking result?

Top score:, 44.50556

Q4. Filtering
Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

How do I debug a docker container?
How do I copy files from a different folder into docker container’s working directory?
How do Lambda container images work?
How can I annotate a graph?


In [76]:
query = "How do copy a file to a Docker container?"

### Method:1 Use Minisearch 

In [77]:
def search(query):
    boost = {'question': 4.0 , 'section' : 0.5}

    results = index.search(
        query = query,
        boost_dict = boost,
        filter_dict = {'course' : 'machine-learning-zoomcamp'},
        num_results = 3
    )

    return results 

In [78]:
search(query)



[{'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy file

***Question*** : What's the 3rd question returned by the search engine?

**Answer** :  'How do I debug a docker container?'

### Method:2 Use Elasticsearch 

In [81]:
def elastic_search(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section^0.5"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index= index_name, body= search_query)

    result_doc = []

    for hit in response['hits']['hits']:
        result_doc.append(hit['_source'])

    return result_doc

In [82]:
elastic_search(query)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

***Question*** : What's the 3rd question returned by the search engine?

**Answer** :  'How do I copy files from a different folder into docker container’s working directory?'

Q5. Building a prompt
Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

context_template = """
Q: {question}
A: {text}
""".strip()
Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
What's the length of the resulting prompt? (use the len function)

946
1446
1946
2446

In [87]:

def build_prompt(query, results):
    prompt_template = """ 
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.


QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt 

In [96]:
build_prompt(query,results)

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\n\nQUESTION: How do copy a file to a Docker container?\n\nCONTEXT:\nsection: 5. Deploying Machine Learning Models\nquestion: How do I debug a docker container?\nanswer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nsection: 10. Kubernetes and TensorFlow Serving\nquestion: Kubernetes-dashboard\nanswer: Deploy and Access the Kubernetes Dashboard\nLuke\n\nsection: 5. Deploying Machine Learning Models\nquestion: How do I copy files from a different folder into docker container’s working directory?\nanswer: You can copy files from your local machine

In [88]:
def llm(prompt):
    response = client.chat.completions.create(
        model= 'llama3-70b-8192',
        messages= [{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [92]:

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [93]:
rag(query)

'To copy a file to a Docker container, you can use the `docker cp` command. The basic syntax is:\n\n`docker cp /path/to/local/file container_id:/path/in/container`\n\nReplace `/path/to/local/file` with the path to the file you want to copy, `container_id` with the ID of the running Docker container, and `/path/in/container` with the path where you want to copy the file in the container.'

In [95]:
search_results = elastic_search(query)

print(len(search_results))

print(len(build_prompt(query, search_results)))

3
1621


Q6. Tokens
When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

pip install tiktoken
Let's calculate the number of tokens in our query:

encoding = tiktoken.encoding_for_model("gpt-4o")
Use the encode function. How many tokens does our prompt have?

120
220
320
420
Note: to decode back a token into a word, you can use the decode_single_token_bytes function:

encoding.decode_single_token_bytes(63842)
